In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)

from IPython.utils import capture
from func.env import setProxyCLI,proxyWget
cb=setProxyCLI()
proxy=cb['proxy']
region=cb['region']
proxyURL=cb['proxyURL']

#@markdown # Dependencies

import os
import subprocess

# 检查 accelerate 库是否已经安装
libraries_to_install = [
    "accelerate==0.12.0",
]

tools_to_install = [
    
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

    
# 定义下载和解压依赖函数
def download_and_unzip_dependencies():
    %cd /openbayes/home/content/
    zip_filename = 'python3.8-dist-packages-extreme.7z'

    # if not os.path.exists(f"/openbayes/home/content/{zip_filename}"):
    #     !$proxy && wget "https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies.{i}"
    #     !mv "Dependencies.{i}" $zip_filename
    # print('Dependencies的7z都已经在本地了，无需下载')

    # 检查 7z 是否已经解压
    unzip_result_exist = os.path.exists("/openbayes/home/content/usr/local/lib/python3.8/dist-packages")
    if not unzip_result_exist:
        # 对7z文件进行解压
        try:
            result = subprocess.run(['7z', 'x', '-y', zip_filename], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if result.returncode == 0:
                print('7z解压命令执行成功')
            else:
                print('7z解压命令执行失败:', result.stderr)
                print('正准备重新下载')
                # 删除
                !rm -rf $zip_filename
                # 重新下载
                download_and_unzip_dependencies()
        except Exception as e:
            print('7z解压命令执行失败:', e)

    else:
        print('7z已经解压')
        
# 执行下载和解压依赖
download_and_unzip_dependencies()
        
# 检查 /usr/local/lib/python3.8/dist-packages 目录是否已经存在
dist_packages_exist = os.path.exists("/usr/local/lib/python3.8/dist-packages")

# 如果 /usr/local/lib/python3.8/dist-packages 目录不存在，则复制
if not dist_packages_exist:
    !cp -r /openbayes/home/content/usr/local/lib/python3.8/dist-packages /usr/local/lib/python3.8/
    # 复制成功后，可以清空解压出来的文件
    !rm -rf /openbayes/home/content/usr

print('[1;32mDONE !\033[0m')